In [18]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import scipy.sparse as sparse
import sklearn.metrics as metrics

In [19]:
testdata = pd.read_csv("Test_Train_Sets/All_test.csv")
traindata = pd.read_csv("Test_Train_Sets/All_train.csv")
valdata = pd.read_csv("Test_Train_Sets/All_validate.csv")

In [20]:
usefulcol = ['Title','ID','Class','Category','Author','ISBN','Count','Address1','Address2','Publisher']
dftrain = traindata[usefulcol]
dftest = testdata[usefulcol]
dfval = valdata[usefulcol]

In [21]:
title_train = np.unique(dftrain['Title'].values)
author_train = np.unique(dftrain['Author'].values)
publisher_train = np.unique(dftrain['Publisher'].values)
title_train_dict = {i:t for i,t in enumerate(title_train)}
author_train_dict = {i:t for i,t in enumerate(author_train)}
publisher_train_dict = {i:t for i,t in enumerate(publisher_train)}
print title_train.shape[0],author_train.shape[0],publisher_train.shape[0]

19148 13581 2165


In [22]:
#dataframe of all books (no duplicates)
dfbook = dftrain[['Author','Title','Publisher','ISBN']]
dfbook = dfbook[~dfbook.duplicated()]
dfbook = dfbook.reset_index()
dfbook.drop('index', axis=1, inplace=True)
dfbook

,Author,Title,Publisher,ISBN
0,<김경진> 등저,데프콘 한미전쟁 세트,씨앗을뿌리는사람,9.788956e+12
1,<베로니카 로스> 저/<이수현> 역,다이버전트,은행나무,9.788957e+12
2,<김한철> 저,"스피라, 세계를 향한 영혼의 승부",랜덤하우스코리아,9.788926e+12
3,<루이스 캐럴> 저/<김양미> 역/<김민지> 그림,이상한 나라의 앨리스,인디고(글담),9.788993e+12
4,<어니스트 헤밍웨이> 저/<이종인> 역,노인과 바다,열린책들,9.788933e+12
5,조지프 캠벨,신의 가면 3,까치(까치글방),9.788973e+12
6,조지프 캠벨,신의 가면 2,까치(까치글방),9.788973e+12
7,조지프 캠벨 저/정영목 역,신의 가면 4,까치(까치글방),9.788973e+12
8,<최진석> 저,노자의 목소리로 듣는 도덕경,소나무,9.788971e+12
9,<조지프 캠벨> 저/<노혜숙> 역,"블리스, 내 인생의 신화를 찾아서",아니마,9.788997e+12


In [ ]:
Xauthor = np.loadtxt('Xauthor.txt')

In [ ]:
Xpublisher = np.loadtxt('Xpublisher.txt')

In [ ]:
simmat_author = Xauthor.dot(Xauthor.T)
simmat_publisher = Xpublisher.dot(Xpublisher.T)